In [46]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm
import json
import os

In [47]:
cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{%22access_token%22:%22eyXqm3C1WZOV4xJQKsTYcjgDrwzu6fh8%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlohtdv.3.2.5',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlocds8.0.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605974420.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605974434.0',
    '_ga': 'GA1.1.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzlzizqzlzqzjzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkznzhzhzkzdzhzdzizlzjzmzqzkznzhzhzkzdzizlzjzmzqzkznzhzhzkzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605974411%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjIncludedInPageviewSample': '1',
    '_hjAbsoluteSessionInProgress': '0',
    '_hjIncludedInSessionSample': '1',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605976041662',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    '_gat': '1',
    'cto_bundle': 'i6f48l9NVXNkQmJ6aEVLcXNqbHdjcVZoQ0k2clladUF2N2xjZzJ1cjR6WG43UTVaRmglMkZXWUdtRnJTNHZRbmQ4SDAlMkZwRFhqQnppRHFxJTJCSEozZXBqRFM4ZHVxUjQ2TmklMkJIcnhJd3luZXpJSnBpcE1nJTNE',
    'TIKI_RECENTLYVIEWED': '58259141',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.3',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi-VN,vi;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://tiki.vn/lemme-dam-so-mi-tay-ngan-fom-rong-xe-ta-p199579469.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.216422_Y.1798742_Z.3604051_CN.AUTO---LEMME-%C4%90am-so-mi-tay-ngan-fom-rong-xe-ta---2023%2F02%2F01-08%3A32%3A01&itm_medium=CPC&itm_source=tiki-ads&spid=199579511',
    'x-guest-token': 'eyXqm3C1WZOV4xJQKsTYcjgDrwzu6fh8',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = (
    ('platform', 'web'),
    ('spid', 187266106),
    ('version', 3)
    #('include', 'tag,images,gallery,promotions,badges,stock_item,variants,product_links,discount_tag,ranks,breadcrumbs,top_features,cta_desktop'),
)

In [48]:
def parser_product(json_data):
    d = {}
    try:
        d['id'] = json_data.get('id')
        d['sku'] = json_data.get('sku')
        d['name'] = json_data.get('name')
        d['short_description'] = json_data.get('short_description')
        d['price'] = json_data.get('price')
        d['list_price'] = json_data.get('list_price')
        d['discount'] = json_data.get('discount')
        d['discount_rate'] = json_data.get('discount_rate')
        d['review_count'] = json_data.get('review_count')
        d['rating_average'] = json_data.get('rating_average')
        d['inventory_status'] = json_data.get('inventory_status')
        
        # Lấy số lượng đã bán
        quantity_sold = json_data.get('quantity_sold')
        if quantity_sold:
            d['quantity_sold'] = quantity_sold.get('value')
        else:
            d['quantity_sold'] = None
        
        # Lấy brand
        brand = json_data.get('brand')
        if brand:
            d['brand_id'] = brand.get('id')
            d['brand_name'] = brand.get('name')
        else:
            d['brand_id'] = None
            d['brand_name'] = None
            
        # Lấy spid
        short_url = json_data.get('short_url')
        split_url = short_url.split('=')
        d['spid'] = split_url[-1]
    
        # Lấy seller_id
        seller = json_data.get('current_seller')
        if seller:
            d['seller_id'] = seller.get('id')
        else:
            d['seller_id'] = None
        
    except Exception as e:
        print(f"Error parsing product: {e}")
    return d

In [49]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

df_id = pd.read_csv(os.path.join(parent_dir, 'Data\product_id_ncds.csv'))
p_ids = df_id.id.to_list()

result = []

error_list = []

progress_bar = tqdm(total=len(p_ids), desc=f'Crawling Pages...', position=0)

for index, pid in enumerate(p_ids):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params, cookies=cookies)
    if response.status_code == 200:
        try:
            product_data = response.json()
            result.append(parser_product(product_data))
        except json.JSONDecodeError as json_error:
            error_list.append({'id': pid, 'error': str(json_error)})
    else:
        print(f"Failed to crawl data for product {pid}. Status code: {response.status_code}")
    progress_bar.update(1)
    # time.sleep(random.randint(2,5))
    
progress_bar.close()

Crawling Pages...:   0%|          | 0/2030 [00:00<?, ?it/s]

Crawling Pages...: 100%|██████████| 2030/2030 [09:17<00:00,  3.64it/s]


In [50]:
order = 'tui-thoi-trang-nu'

df_product = pd.DataFrame(result)
df_product.to_csv(os.path.join(parent_dir, f'Data\{order}.csv'), index=False, encoding='utf-8')
print(f"Crawled {order}")

print(json.dumps(error_list[:5], indent=4))
df_error_products = pd.DataFrame(error_list)
df_error_products.to_csv(os.path.join(parent_dir, f'Data\{order}_error_product_list.csv'), index=False, encoding='utf-8')

Crawled tui-thoi-trang-nu
[
    {
        "id": 155169078,
        "error": "Expecting value: line 1 column 1 (char 0)"
    },
    {
        "id": 93831393,
        "error": "Expecting value: line 1 column 1 (char 0)"
    },
    {
        "id": 108155133,
        "error": "Expecting value: line 1 column 1 (char 0)"
    },
    {
        "id": 118403624,
        "error": "Expecting value: line 1 column 1 (char 0)"
    },
    {
        "id": 185468208,
        "error": "Expecting value: line 1 column 1 (char 0)"
    }
]
